In [65]:
from gradio_client import Client
from pathlib import Path
import json
import pandas as pd

In [107]:
path_data = Path("../data")

In [ ]:
client = Client("https://felixz-open-llm-leaderboard.hf.space/")
json_data = client.predict("","", api_name='/predict')

with open(json_data, 'r') as file:
    file_data = file.read()
    data = json.loads(file_data)
    df = pd.DataFrame(data['data'], columns=data['headers'])
    df.drop(columns=['Model'], inplace=True)
    df.to_json(path_data / 'HF-open-llm-leaderboard_20231114.json', orient='records', indent=4)

In [113]:
import os
import pandas as pd
import plotly.express as px

# Function to read a JSON file into a DataFrame
def read_json_file(file_path):
    return pd.read_json(file_path)

# List of JSON file paths (replace with your file paths)
json_files = [os.path.join(path_data, file) for file in os.listdir(path_data)]

# Read each file and store in a list of DataFrames
dataframes = [read_json_file(file) for file in json_files]

# Extract and count models in each DataFrame
model_counts = []
for i, df in enumerate(dataframes):
    count = df['model'].value_counts().reset_index()
    count.columns = ['model', 'count']
    count['file'] = f'file_{i+1}'  # Naming files as file_1, file_2, etc.
    model_counts.append(count)

# Combine all counts into a single DataFrame
combined_df = pd.concat(model_counts)

# Create a bubble chart
fig = px.scatter(combined_df, x="model", y="file", size="count", 
                 size_max=60, title="Model Coverage Across Different Files")

# Show the plot
fig.show()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 554: invalid start byte

In [115]:
for file in json_files:
    print(file)
    read_json_file(file)

../data/AlpacaEval-leaderboard-claude-20231109.json
../data/Opencompass-llm-leaderboard-english_benchmark-language-20231114.json
../data/HF-SEED-Bench-Leaderboard-20231110.json
../data/LucyEval-leaderboard-20230814.json
../data/Opencompass-llm-leaderboard-chinese_benchmark-overall-20231114.json
../data/HF-chatbot-arena-leaderboard-20231108.json
../data/Opencompress-botchat-20231102.json
../data/.DS_Store


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 554: invalid start byte

In [106]:
# df = pd.read_csv('b.csv')
# df.to_json(path_data / 'Hallucination-leaderboard-20231114.json', orient='records', indent=4)